In [4]:
import os
import json
import string
import pickle 
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm,tnrange
from collections import Counter
#nltk.download("punkt")
#nltk.download("stopwords")

In [5]:
class InvertedIndex():
    def __init__(self):
        self.DFpostings={}                #Dictionary contains Posting List for each words 
        self.termsInFile=[]               #List contains all words in a single file

    def stripSpecialChar(self,text):
        return ''.join(ch for ch in text if ch.isalnum() and not ch.isdigit() and ch not in string.punctuation)

    def preProcess(self,text):
        stemmer = SnowballStemmer("english")
        stopWords = set(stopwords.words('english'))

        text = text.lower()                                     # convert all text to lower case
        text_tokens = word_tokenize(text)                       # tokenizing the text

        stemmedWords = list([stemmer.stem(word) for word in text_tokens])   # Stemming 
        validTokens = [i for i in stemmedWords if i not in stopWords]       # Removing Stop Words

        validTokens = [self.stripSpecialChar(x) for x in validTokens]   # stripping special characters
        validTokens = [x for x in validTokens if len(x) > 1]    # Choosing only words which has length > 1
        return validTokens, set(validTokens)

    def indexFile(self,file,fileId):
        '''
        Creates Index for each File
        '''
        tokens, setTokens = self.preProcess(file)
        self.termsInFile.append(tokens)
        for i in setTokens:
            if i in self.DFpostings:
                self.DFpostings[i].append(fileId)
            else:
                self.DFpostings[i] = [fileId]
                
    def save(self,file):
        '''
        Save the index to a file locally
        '''
        json.dump(self.DFpostings, open("DFPostings"+file, "w"))
        json.dump(self.termsInFile, open("TermsInfile"+file, "w"))

In [6]:
allFiles = os.walk("recipes_folder")
filePaths = []
for i in allFiles:
    for j in i[2]:
        filePath = i[0] + "/" + j
        filePaths.append(filePath)

json.dump(filePaths, open("Final_mapping.json", "w"))

indexFile = InvertedIndex()
indexTitle = InvertedIndex()

for i,filePath in enumerate(tqdm(filePaths)):
    file = open(filePath, encoding="utf8")
    readTitle = file.readline().replace('\n', ' ')
    readFile = file.read().replace('\n', ' ')
    
    file.close()
    indexFile.indexFile(readFile, i)
    indexTitle.indexFile(readTitle, i)
    
#indexFile.save("File.txt")
#indexTitle.save("Title.txt")   

# Saving the File
serializedFile = pickle.dumps(indexFile)
filename = 'ProjectFile.obj'
with open(filename,'wb') as file_object:
    file_object.write(serializedFile)
    
# Saving the Title 
serializedTitle = pickle.dumps(indexTitle)
filename = 'ProjectTitle.obj'
with open(filename,'wb') as file_object:
    file_object.write(serializedTitle)

0it [00:00, ?it/s]